In [1]:
import requests
import json
import sqlite3
from datetime import datetime

In [2]:
def setup_database():
    conn = sqlite3.connect("live_wikipedia.db")
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS wiki_edits (
            timestamp TEXT,
            user_name TEXT,
            page_title TEXT,
            is_bot BOOLEAN,
            edit_length_change INTEGER,
            wikipedia_domain TEXT
        )
    ''')
    conn.commit()
    return conn

In [3]:
def stream_wikipedia_data():
    print("🔌 Connecting to Wikipedia's Live Data Firehose...", flush=True)
    
    url = 'https://stream.wikimedia.org/v2/stream/recentchange'
    
    # NEW: We are adding a "name tag" so Wikipedia doesn't block us
    headers = {
        'User-Agent': 'DataEngineeringPoC/1.0 (Python/requests)'
    }
    
    conn = setup_database()
    cursor = conn.cursor()
    
    print("📡 Connected! Listening for live global edits...\n", flush=True)
    print("-" * 70, flush=True)
    
    try:
        # NEW: Added the headers=headers parameter here
        with requests.get(url, headers=headers, stream=True, timeout=None) as response:
            
            # Let's print the status code to make sure Wikipedia let us in!
            print(f"HTTP Status Code: {response.status_code} (Should be 200)", flush=True)
            
            if response.status_code != 200:
                print("❌ Connection refused by Wikipedia. Stopping.")
                return

            for line in response.iter_lines(chunk_size=1):
                if line:
                    decoded_line = line.decode('utf-8')
                    
                    if decoded_line.startswith('data: '):
                        try:
                            json_data = json.loads(decoded_line[6:])
                            
                            # Filter for actual Wikipedia article edits
                            if json_data.get('type') == 'edit' and json_data.get('namespace') == 0:
                                
                                user = json_data.get('user', 'Unknown')
                                title = json_data.get('title', 'Unknown')
                                is_bot = json_data.get('bot', False)
                                domain = json_data.get('meta', {}).get('domain', 'Unknown')
                                
                                old_len = json_data.get('length', {}).get('old', 0)
                                new_len = json_data.get('length', {}).get('new', 0)
                                length_diff = new_len - old_len
                                
                                current_time = datetime.now().strftime("%H:%M:%S")
                                
                                cursor.execute('''
                                    INSERT INTO wiki_edits (timestamp, user_name, page_title, is_bot, edit_length_change, wikipedia_domain)
                                    VALUES (?, ?, ?, ?, ?, ?)
                                ''', (current_time, user, title, is_bot, length_diff, domain))
                                conn.commit()
                                
                                bot_status = "🤖 BOT" if is_bot else "👤 HUMAN"
                                print(f"[{current_time}] {bot_status} edited: '{title}' ({length_diff} chars) on {domain}", flush=True)
                                
                        except json.JSONDecodeError:
                            continue 
                            
    except KeyboardInterrupt:
        print("\n🛑 Pipeline stopped by user. Closing database connection.", flush=True)
        conn.close()
    except Exception as e:
        print(f"\n❌ Pipeline crashed: {e}", flush=True)
        conn.close()

        
if __name__ == "__main__":
    stream_wikipedia_data()

🔌 Connecting to Wikipedia's Live Data Firehose...
📡 Connected! Listening for live global edits...

----------------------------------------------------------------------
HTTP Status Code: 200 (Should be 200)
[17:00:12] 👤 HUMAN edited: 'Au pays du scalp' (0 chars) on fr.wikipedia.org
[17:00:12] 👤 HUMAN edited: '-onjë' (0 chars) on en.wiktionary.org
[17:00:12] 👤 HUMAN edited: 'Q130790289' (1071 chars) on www.wikidata.org
[17:00:12] 👤 HUMAN edited: 'Q133412539' (1071 chars) on www.wikidata.org
[17:00:13] 🤖 BOT edited: 'Sprachförderung' (10 chars) on de.wiktionary.org
[17:00:13] 🤖 BOT edited: 'Q114826790' (0 chars) on www.wikidata.org
[17:00:13] 👤 HUMAN edited: 'Middle Russian language' (-3 chars) on en.wikipedia.org
[17:00:13] 👤 HUMAN edited: 'Photokeratitis' (-53 chars) on en.wikipedia.org
[17:00:13] 👤 HUMAN edited: 'Q130797369' (1071 chars) on www.wikidata.org
[17:00:14] 👤 HUMAN edited: 'Q133062038' (1072 chars) on www.wikidata.org
[17:00:14] 👤 HUMAN edited: 'Q107249654' (-161 chars) on

In [4]:
import sqlite3
import pandas as pd

def analyze_live_data():
    print("📊 Analyzing Wikipedia Stream Data...\n")
    
    # Connect to the database we just populated
    conn = sqlite3.connect("live_wikipedia.db")
    
    # 1. How many total edits did we capture?
    total_edits = pd.read_sql_query("SELECT COUNT(*) as Total_Edits FROM wiki_edits", conn)
    print(f"Total Edits Captured: {total_edits['Total_Edits'][0]}")
    print("-" * 40)
    
    # 2. Human vs. Bot Ratio
    bot_ratio = pd.read_sql_query('''
        SELECT 
            CASE WHEN is_bot = 1 THEN 'Bots 🤖' ELSE 'Humans 👤' END as User_Type,
            COUNT(*) as Edit_Count
        FROM wiki_edits
        GROUP BY is_bot
    ''', conn)
    print("Who is making the edits?")
    print(bot_ratio.to_string(index=False))
    print("-" * 40)
    
    # 3. What were the top 5 most heavily edited pages?
    # We use ABS() to count the total volume of text changed (added or removed)
    top_pages = pd.read_sql_query('''
        SELECT page_title, SUM(ABS(edit_length_change)) as Total_Chars_Changed
        FROM wiki_edits
        GROUP BY page_title
        ORDER BY Total_Chars_Changed DESC
        LIMIT 5
    ''', conn)
    
    print("Top 5 Most Heavily Modified Pages:")
    print(top_pages.to_string(index=False))
    
    conn.close()

# Run the analysis
analyze_live_data()

📊 Analyzing Wikipedia Stream Data...

Total Edits Captured: 3856
----------------------------------------
Who is making the edits?
User_Type  Edit_Count
 Humans 👤        3164
   Bots 🤖         692
----------------------------------------
Top 5 Most Heavily Modified Pages:
                               page_title  Total_Chars_Changed
                                  America                74784
                     কেলাসবিজ্ঞান ডেটাবেস                59878
                                   حضرموت                53943
                               Q138481572                34885
2022 Punjab Legislative Assembly election                23227
